In [2]:
# !pip install --upgrade pip
# !pip install huggingface_hub
# !pip install transformers
# !pip install datasets
# !pip install peft
# !pip install bitsandbytes
# !pip install -U typing_extensions
# !pip install -U transformers datasets peft accelerate bitsandbytes

In [ ]:
from huggingface_hub import login

login(token='')

In [4]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer,
    DataCollatorForLanguageModeling, BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [5]:
# 1. 모델 이름
model_name = "mistralai/Mistral-7B-v0.1"

In [6]:
# 2. 4bit 양자화 설정

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=True,
    llm_int8_enable_fp32_cpu_offload=True
)


In [7]:
# 3. 토크나이저 및 모델 로드
# tokenizer에 pad_token 지정 필수
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    offload_folder="./offload"  # 디스크 캐시 위치
)

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [8]:
# 4. QLoRA 적용을 위한 PEFT 설정
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [9]:
# 5. 데이터 로드 및 전처리
dataset = load_dataset("json", data_files="./dog_lm_dataset.json")

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset["train"].map(tokenize, batched=True, remove_columns=["text"])
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/83 [00:00<?, ? examples/s]

In [10]:
# 6. 학습 인자 설정
training_args = TrainingArguments(
    output_dir="./qlora_mistral_dog_model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=20,
    save_strategy="epoch",
    save_total_limit=1,
    fp16=False,
    bf16=True,
    report_to="none"
)

In [11]:
# 7. Trainer 실행
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=collator
)

trainer.train()

/tmp/ipykernel_454/1741438243.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
20,1.243200


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=30, training_loss=1.22690060933431, metrics={'train_runtime': 154.0153, 'train_samples_per_second': 1.617, 'train_steps_per_second': 0.195, 'total_flos': 5026515863470080.0, 'train_loss': 1.22690060933431, 'epoch': 2.761904761904762})

In [12]:
# 8. 어댑터 저장 (base model은 저장 안함)
model.save_pretrained("./qlora_mistral_dog_model")
tokenizer.save_pretrained("./qlora_mistral_dog_model")

('./qlora_mistral_dog_model/tokenizer_config.json',
 './qlora_mistral_dog_model/special_tokens_map.json',
 './qlora_mistral_dog_model/tokenizer.json')

In [13]:
# import torch
# torch.cuda.empty_cache()

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel

# 1. base 모델 및 tokenizer 로드
base_model_name = "mistralai/Mistral-7B-v0.1"
adapter_path = "./qlora_mistral_dog_model"

tokenizer = AutoTokenizer.from_pretrained(adapter_path)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="auto",
    offload_folder="./offload",
)

# 2. 어댑터 모델 적용
model = PeftModel.from_pretrained(base_model, adapter_path)

# 3. pipeline 생성
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

In [39]:
default = pipeline("text-generation", model=base_model, tokenizer=tokenizer)

Device set to use cuda:0


In [46]:
prompt = (
   "You are an expert in the Basset Hound breed."
"Tell me about the Basset Hound breed."
"Please answer including the characteristics of the Basset Hound breed."
)

output = pipe(prompt, max_new_tokens=1000, do_sample=True, temperature=0.3)[0]["generated_text"]
print("\n✅ 생성 결과:\n", output)

output = default(prompt, max_new_tokens=1000, do_sample=True, temperature=0.3)[0]["generated_text"]
print("\n✅ 생성 결과:\n", output)


✅ 생성 결과:
 You are an expert in the Basset Hound breed.Tell me about the Basset Hound breed.Please answer including the characteristics of the Basset Hound breed.

Basset Hounds are a type of dog that was originally bred in France. They are known for their short, stocky bodies and long ears. Basset Hounds are typically very friendly and affectionate dogs, and they make great companions. They are also very good at tracking and hunting, and they have been used for these purposes for centuries.

Basset Hounds are a type of dog that was originally bred in France. They are known for their short, stocky bodies and long ears. Basset Hounds are typically very friendly and affectionate dogs, and they make great companions. They are also very good at tracking and hunting, and they have been used for these purposes for centuries.

Basset Hounds are a type of dog that was originally bred in France. They are known for their short, stocky bodies and long ears. Basset Hounds are typically very friend